In [2]:
from graph_tool.all import *
import graph_tool.all as gt
import numpy as np
import random
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# モジュール付きネットワーク生成関数
def module_network(n, m, m0, beta, seed=None):

    # 乱数生成のシードを設定可能
    if seed is not None:
        np.random.seed(seed=seed)

    initial_nodes = m + 1  # 初期ノード数:m + 1

    g = gt.complete_graph(initial_nodes, directed = False)  # 初期の完全グラフを生成

    initial_edges = list(g.edges()) # 初期のリンクの一覧を生成

    node_list = list(range(initial_nodes)) #初期のノードの一覧を生成

    module_number = g.new_vertex_property("int")  # モジュール番号を格納する新しいプロパティ
    node_id_property = g.new_vertex_property("int")  # ノードIDを格納する新しいプロパティ

    # 初期ノードのNode_IDを手動で割り当てる
    for i in range(initial_nodes):
        node_id_property[g.vertex(i)] = i

    node_id = initial_nodes - 1  # 次のノードID

    # 新規ノードを追加し、リンクをべき乗の値を参照して追加していく
    for i in range(initial_nodes, n):

        # 現在のネットワークの各ノードの次数を計算しk_array_before配列に保管
        K_array_before = np.array([v.out_degree() for v in g.vertices() if v.out_degree() != 0])  # 次数の配列 全頂点の次数（次数が0のノードを除外）

        # べき乗の値によって、K_array_beforeの中身を再計算する
        if beta == 0 or beta == 1:
            K_array = K_array_before**beta
        elif beta == 1/2:
            K_array = np.sqrt(K_array_before)
        else:
            ipa = abs(beta)
            K_array = 1 / np.power(K_array_before, ipa, dtype=np.float64)

        g.add_vertex()  # 新規ノードを追加

        for j in range(m):
            # 次数に比例した確率ベクトルを生成
            s = np.sum(K_array)
            parray = K_array / s
            parray /= np.sum(parray)  # 確率の合計を1.0に調整

            # 次数に比例した確率で既存のノードを選び、エッジとする
            cumulative_probs = np.cumsum(parray)
            r = random.random()
            new = node_list[np.searchsorted(cumulative_probs, r)]
            g.add_edge(g.vertex(i), g.vertex(new))
            
            K_array_before[i] += 1
            K_array_before[new] += 1

            K_array[node_list.index(new)] = 0.

        node_list.append(i)  # 新規ノードをノードリストに追加する
        node_id += 1  # 新しいノードIDを割り当てる
        node_id_property[g.vertex(i)] = node_id  # ノードIDをプロパティに格納

    random.shuffle(node_list)  # ノードリスト内の要素をランダムに並び替えることで、モジュール番号がランダムに選ばれるようにする
    nodes_per_module = n // m0  # 各モジュール毎に割り当てるノード数を求める

    for node_index, node in enumerate(node_list):

        assigned_module = (node_index // nodes_per_module) + 1  # 各ノードにモジュール番号を割り当てる
        module_number[g.vertex(node)] = assigned_module  # 各ノードのモジュール番号をプロパティに格納する

    g.vertex_properties["module_number"] = module_number  # モジュール番号のプロパティを設定
    g.vertex_properties["node_id"] = node_id_property  # ノードIDのプロパティを設定

    return g

In [4]:
#チェーン構造を解消するためのリワイヤリングコード
def rewire_network(g, rewiring_times):
    #print("random_rewire開始")
    gt.random_rewire(g, model = "configuration", n_iter = rewiring_times, edge_sweep=True)
    #print("random_rewire終了")
    return g

# Define function to check sum_u
def check_sum_u(g, target_sum):
    l = gt.label_largest_component(g)
    u = gt.GraphView(g, vfilt=l)
    sum_u = u.num_vertices()
    return sum_u == target_sum

In [5]:
Node = 10000

m = 2

times = 100

M_list = [5,10,20,50,100,200]

beta_list = [1,1/2,0,-1,-3,-5,-10,-100]

In [1]:
for M in M_list:
    for beta in beta_list:
        graphs = []
        for _ in tqdm(range(times)):
            while True:
                g = module_network(Node, m, M, beta, seed=None)
                g = rewire_network(g, Node)
                if check_sum_u(g, Node):  # 条件を満たしたらループ脱出
                    break
            graphs.append(g)

        # 保存先ディレクトリを作成
        directory = f"~/o_t_hayashilab/Network_data/graph-tool/modular_network/N={Node}/beta={beta}/w=0.00/Mo={M}"
        directory = os.path.expanduser(directory)
        os.makedirs(directory, exist_ok=True)

        # グラフを保存
        for i, graph in enumerate(graphs):
            g = graphs
            filename = os.path.join(directory, f"{i}.gt.gz")
            g.save(filename)

NameError: name 'M_list' is not defined